# ** RNN **

일반적인 딥러닝 모델로는 대응하기 어려운 시계열 데이터를 다루는 데 사용하는 모델을 **순환 신경망(Recurrent Neural Network)**라고 한다. 순환 신경망은 규칙성이나 패턴이 있는  <U>시계열 데이터</U>를 학습하여, 미지의 새로운 시계열 데이터가 주어졌을 때 그 데이터의 미래 상태를 예측할 수 있다.

## ** Tutorial 0 : What is the RNN?**
#### ** RNN 사용법을 간단히 익혀보자. **

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.python.framework import ops

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from IPython.display import Image

In [2]:
# H -> E -> L -> L -> O
data = ['H', 'E', 'L', 'L', 'O']

label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder()

# 1. String -> int label
labels = label_encoder.fit_transform(data)
# 2. Reshape input for onehot encoder
labels = labels.reshape(len(labels), 1)
# 3. Int label-> one hot vector
onehot_vec = onehot_encoder.fit_transform(labels)
# 4. Print and check the result.
inputs = onehot_vec.toarray()

In [3]:
hidden_size = 2
batch_size = 1

In [4]:
# Input data Shape : (Data size, 5, 4)
# 5 means the number of time series. (H -> E -> L -> L -> O)
# 4 means the dimension of the time data. (H : [1, 0, 0, 0], E : [0, 1, 0, 0], ...)
x = np.array(inputs.reshape(1, 5, 4), dtype=np.float32)

In [5]:
# Define the BasicRNNCell
rnn_cell = tf.contrib.rnn.BasicRNNCell(hidden_size)
initial_state = rnn_cell.zero_state(batch_size, dtype=tf.float32)
# Dynamic_rnn
outputs, state = tf.nn.dynamic_rnn(rnn_cell, x, initial_state=initial_state, dtype=tf.float32)

In [6]:
sess = tf.InteractiveSession()

In [7]:
sess.run(tf.global_variables_initializer())

In [8]:
result = outputs.eval()

print(result.shape)

# Output
print(result)

# 마지막 output
print(result[:, -1, :])

(1, 5, 2)
[[[-0.05091725 -0.14921738]
  [-0.6142292  -0.18411921]
  [ 0.29054296 -0.01310057]
  [ 0.5645038   0.38256508]
  [ 0.32585782 -0.19290096]]]
[[ 0.32585782 -0.19290096]]


> Result를 해석해보자.
 - 5의 의미 : 1 time series를 구성하는 time state data의 갯수(H,E,L,L,O). 이것은 Time series를 구성하는 hidden cell의 output 갯수와도 동일
 - 2의 의미 : Hidden cell output의 차원수를 의미

<img src="image/RNN2.png" width="800" height="700">

#### ** Dynamic_rnn 을 사용하지 않고, BasicRNNCell만 사용할 경우는 각 시간에 대한 state와 cell output을 구하는 코드를 작성해야 한다. **

In [9]:
# Rnn_cell의 input으로는 tensor가 사용된다.
tf_x = ops.convert_to_tensor(x)

cell_outputs = []
states = []

state = initial_state
for i in range(5):
    cell_output, state = rnn_cell(tf_x[:, i, :], state)
    cell_outputs.append(cell_output)
    states.append(state)

In [10]:
# Dynamic_rnn의 마지막 output과 동일함을 알 수 있다.
cell_outputs[-1].eval(session=sess)

array([[ 0.32585782, -0.19290096]], dtype=float32)

In [11]:
states[-1].eval(session=sess)

array([[ 0.32585782, -0.19290096]], dtype=float32)